<a href="https://colab.research.google.com/github/samitha278/multilayer-perceptron-char-lm/blob/main/mlp_char_lm.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [2]:
import torch
import torch.nn.functional as F
import matplotlib.pyplot as plt

In [3]:
from google.colab import files

uploaded = files.upload()


Saving names.txt to names (1).txt


In [4]:
words = open("names.txt","r").read().splitlines()

In [5]:
chars = sorted(list(set("".join(words))))
stoi = {s:i+1 for i,s in enumerate(chars)}
stoi["."] = 0
itos = {i:s for s,i in stoi.items()}
itos

{1: 'a',
 2: 'b',
 3: 'c',
 4: 'd',
 5: 'e',
 6: 'f',
 7: 'g',
 8: 'h',
 9: 'i',
 10: 'j',
 11: 'k',
 12: 'l',
 13: 'm',
 14: 'n',
 15: 'o',
 16: 'p',
 17: 'q',
 18: 'r',
 19: 's',
 20: 't',
 21: 'u',
 22: 'v',
 23: 'w',
 24: 'x',
 25: 'y',
 26: 'z',
 0: '.'}

### Building dataset

In [6]:
block_size = 3


xs = []
ys = []


context = [0]*block_size



for w in words:

  for ch in w + '.':

    xs.append(context)
    next = stoi[ch]
    ys.append(next)


    context = context[1:] + [next]


x = torch.tensor(xs)
y = torch.tensor(ys)

In [7]:
x.shape,y.shape

(torch.Size([228146, 3]), torch.Size([228146]))

In [8]:
x[:5]

tensor([[ 0,  0,  0],
        [ 0,  0,  5],
        [ 0,  5, 13],
        [ 5, 13, 13],
        [13, 13,  1]])

### Embedding

In [9]:
C = torch.randn((27,2))
C

tensor([[-0.3267,  1.8026],
        [-1.6643,  0.3641],
        [ 0.1016,  0.0644],
        [-1.0396, -2.8078],
        [-1.7219,  0.7559],
        [ 0.6209,  0.8554],
        [ 0.3442, -0.4181],
        [ 0.4559,  1.2350],
        [-0.7609,  0.4817],
        [-0.1752,  1.4614],
        [-0.1634,  1.1385],
        [ 1.0621,  0.0766],
        [-1.3921,  2.1695],
        [-0.1046, -0.6759],
        [ 0.1310,  0.2420],
        [ 0.9748, -0.1959],
        [ 0.3516, -0.3428],
        [-0.1911,  1.0875],
        [-1.4573,  2.2874],
        [ 0.3889,  0.1198],
        [ 0.1917, -1.0685],
        [-0.2072, -1.0520],
        [ 0.0086, -0.1605],
        [-1.1808,  1.1842],
        [ 0.4114,  1.0143],
        [-1.4451, -0.9445],
        [-1.0122, -0.3389]])

In [10]:
C[5]

tensor([0.6209, 0.8554])

In [11]:
C[torch.tensor([2,3,5,5,5])]

tensor([[ 0.1016,  0.0644],
        [-1.0396, -2.8078],
        [ 0.6209,  0.8554],
        [ 0.6209,  0.8554],
        [ 0.6209,  0.8554]])

In [12]:
C[torch.tensor([[2,3],[3,2],[5,5],[2,5]])]

tensor([[[ 0.1016,  0.0644],
         [-1.0396, -2.8078]],

        [[-1.0396, -2.8078],
         [ 0.1016,  0.0644]],

        [[ 0.6209,  0.8554],
         [ 0.6209,  0.8554]],

        [[ 0.1016,  0.0644],
         [ 0.6209,  0.8554]]])

In [13]:
emb = C[x]
emb.shape

torch.Size([228146, 3, 2])

In [14]:
x[1],C[[0,5]],emb[1]

(tensor([0, 0, 5]),
 tensor([[-0.3267,  1.8026],
         [ 0.6209,  0.8554]]),
 tensor([[-0.3267,  1.8026],
         [-0.3267,  1.8026],
         [ 0.6209,  0.8554]]))

### Preparing input neurons to feed hidden layer

In [15]:
emb[:5]

tensor([[[-0.3267,  1.8026],
         [-0.3267,  1.8026],
         [-0.3267,  1.8026]],

        [[-0.3267,  1.8026],
         [-0.3267,  1.8026],
         [ 0.6209,  0.8554]],

        [[-0.3267,  1.8026],
         [ 0.6209,  0.8554],
         [-0.1046, -0.6759]],

        [[ 0.6209,  0.8554],
         [-0.1046, -0.6759],
         [-0.1046, -0.6759]],

        [[-0.1046, -0.6759],
         [-0.1046, -0.6759],
         [-1.6643,  0.3641]]])

In [16]:
emb[:5,0]

tensor([[-0.3267,  1.8026],
        [-0.3267,  1.8026],
        [-0.3267,  1.8026],
        [ 0.6209,  0.8554],
        [-0.1046, -0.6759]])

In [17]:
torch.cat([emb[:5,0],emb[:5,1],emb[:5,2]],dim=1)

tensor([[-0.3267,  1.8026, -0.3267,  1.8026, -0.3267,  1.8026],
        [-0.3267,  1.8026, -0.3267,  1.8026,  0.6209,  0.8554],
        [-0.3267,  1.8026,  0.6209,  0.8554, -0.1046, -0.6759],
        [ 0.6209,  0.8554, -0.1046, -0.6759, -0.1046, -0.6759],
        [-0.1046, -0.6759, -0.1046, -0.6759, -1.6643,  0.3641]])

In [18]:
torch.cat([emb[:,0],emb[:,1],emb[:,2]],dim=1).shape

torch.Size([228146, 6])

In [19]:
torch.cat(torch.unbind(emb,1),1).shape

torch.Size([228146, 6])

### torch view and storage

In [20]:
a = torch.arange(12)
a

tensor([ 0,  1,  2,  3,  4,  5,  6,  7,  8,  9, 10, 11])

In [21]:
a.storage()

/tmp/ipython-input-214256462.py:1: UserWarning: TypedStorage is deprecated. It will be removed in the future and UntypedStorage will be the only storage class. This should only matter to you if you are using storages directly.  To access UntypedStorage directly, use tensor.untyped_storage() instead of tensor.storage()
  a.storage()


 0
 1
 2
 3
 4
 5
 6
 7
 8
 9
 10
 11
[torch.storage.TypedStorage(dtype=torch.int64, device=cpu) of size 12]

In [22]:
a.view(3,4)

tensor([[ 0,  1,  2,  3],
        [ 4,  5,  6,  7],
        [ 8,  9, 10, 11]])

In [23]:
#only view has been changed memory allocation of a is same as before

# https://blog.ezyang.com/2019/05/pytorch-internals/

a.view(3,4).storage()

 0
 1
 2
 3
 4
 5
 6
 7
 8
 9
 10
 11
[torch.storage.TypedStorage(dtype=torch.int64, device=cpu) of size 12]

### Implementing hidden layer

In [24]:
W1 = torch.randn((6,100))
b1 = torch.randn(100)

In [25]:
h = torch.tanh(emb.view(-1,6) @ W1 + b1)

In [26]:
h.shape , h

(torch.Size([228146, 100]),
 tensor([[ 0.9678, -1.0000, -0.9977,  ..., -0.7191,  0.9954,  0.8426],
         [-0.5085, -1.0000, -0.8353,  ...,  0.7963,  0.9315,  0.8379],
         [-0.9831, -0.9999, -0.9747,  ...,  0.8230,  0.9207, -0.7672],
         ...,
         [-0.8871, -0.5687,  0.5086,  ..., -0.9093, -0.9216, -0.9876],
         [-0.3703, -0.6264,  0.8375,  ..., -0.9918, -0.2505, -0.9677],
         [ 0.9923, -0.9831,  0.7876,  ..., -0.9978,  0.3601,  0.9394]]))

### Output layer

In [27]:
W2 = torch.randn((100,27))
b2 = torch.randn(27)

In [28]:
logits = h @ W2 + b2
logits.shape

torch.Size([228146, 27])

### Softmax

In [29]:
p = logits.exp()
p /= p.sum(1,keepdim=True)

p.shape

torch.Size([228146, 27])

### Loss

In [30]:
loss = -p[torch.arange(p.shape[0]),y].log().mean()
loss

tensor(18.2166)

### Bcckpropagation

In [33]:
W1.grad = None
b1.grad = None
W2.grad = None
b2.grad = None

loss.backward(W1)
loss.backward(b1)
loss.backward(W2)
loss.backward(b2)

RuntimeError: Mismatch in shape: grad_output[0] has a shape of torch.Size([6, 100]) and output[0] has a shape of torch.Size([]).